In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition

In [2]:


from idaes.core import FlowsheetBlock



In [3]:


from idaes.models.unit_models import (PressureChanger,
                                      Mixer,
                                      Separator as Splitter,
                                      Heater,
                                      StoichiometricReactor,
                                      Flash)



In [4]:
from idaes.models.unit_models.pressure_changer import ThermodynamicAssumption
from idaes.core.util.model_statistics import degrees_of_freedom

# Import idaes logger to set output levels
import idaes.logger as idaeslog

In [9]:
import hda_ideal_VLE as thermo_props
import hda_reaction as reaction_props


In [10]:


m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)



In [11]:
m.fs.thermo_params = thermo_props.HDAParameterBlock()
m.fs.reaction_params = reaction_props.HDAReactionParameterBlock(
        property_package=m.fs.thermo_params)


2025-03-25 15:39:23 [WARNING] idaes.core.base.property_meta: DEPRECATED: The property name fug_vap_comp in property metadata is not
a recognized standard property name defined in this PropertySet.
Please refer to IDAES standard names in the IDAES documentation. You
can use the define_custom_properties() rather than the
add_properties() method to define metadata for this property. You can
also use a different property set by calling the define_property_set()
method.  (deprecated in 2.0.0, will be removed in (or after) 3.0.0)
(called from c:\Users\Sara\Desktop\IDAES\hda_ideal_VLE.py:378)
2025-03-25 15:39:23 [WARNING] idaes.core.base.property_meta: DEPRECATED: The property name fug_liq_comp in property metadata is not
a recognized standard property name defined in this PropertySet.
Please refer to IDAES standard names in the IDAES documentation. You
can use the define_custom_properties() rather than the
add_properties() method to define metadata for this property. You can
also use a diffe

In [12]:


m.fs.M101 = Mixer(property_package=m.fs.thermo_params,
                  inlet_list=["toluene_feed", "hydrogen_feed", "vapor_recycle"])

m.fs.H101 = Heater(property_package=m.fs.thermo_params,
                   has_pressure_change=False,
                   has_phase_equilibrium=True)



In [14]:
m.fs.R101 = StoichiometricReactor(property_package=m.fs.thermo_params,
                                    reaction_package=m.fs.reaction_params,
                                    has_heat_of_reaction=True,
                                    has_heat_transfer=True,
                                    has_pressure_change=False)


In [15]:
m.fs.F101 = Flash(property_package=m.fs.thermo_params,
                  has_heat_transfer=True,
                  has_pressure_change=True)

In [16]:


m.fs.S101 = Splitter(property_package=m.fs.thermo_params,
                     ideal_separation=False,
                     outlet_list=["purge", "recycle"])
    

m.fs.C101 = PressureChanger(
            property_package=m.fs.thermo_params,
            compressor=True,
            thermodynamic_assumption=ThermodynamicAssumption.isothermal)
    
m.fs.F102 = Flash(property_package=m.fs.thermo_params,
                  has_heat_transfer=True,
                  has_pressure_change=True)



In [17]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)

In [18]:
#Todo: Connect the H101 outlet to R101 inlet
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)

In [19]:


m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.F101.inlet)
m.fs.s06 = Arc(source=m.fs.F101.vap_outlet, destination=m.fs.S101.inlet)
m.fs.s08 = Arc(source=m.fs.S101.recycle, destination=m.fs.C101.inlet)
m.fs.s09 = Arc(source=m.fs.C101.outlet,
               destination=m.fs.M101.vapor_recycle)
m.fs.s10 = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.F102.inlet)



In [20]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [30]:
import idaes_ui


In [38]:
import idaes_ui
dir(idaes_ui)  # This will list all the available attributes and functions in the idaes_ui package


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'fv']

In [39]:
import idaes_ui.fv
dir(idaes_ui.fv)


['VisualizeResult',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'errors',
 'flowsheet',
 'fsvis',
 'icons',
 'model_server',
 'models',
 'persist',
 'validate_flowsheet',
 'visualize']

In [ ]:
from idaes_ui.fv import visualize  



In [42]:
visualize(m.fs, "My Process Flowsheet")

2025-03-25 16:15:13 [INFO] idaes.idaes_ui.fv.fsvis: Started visualization server
2025-03-25 16:15:13 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'My Process Flowsheet.json' in current directory (c:\Users\Sara\Desktop\IDAES)
2025-03-25 16:15:13 [WARNING] idaes.idaes_ui.fv.flowsheet: Cannot extract state block from Port: name=liq_outlet. Please add Feed & Product blocks with Arcs to show inlet and outlet stream values in the Stream Table
2025-03-25 16:15:13 [WARNING] idaes.idaes_ui.fv.flowsheet: Cannot extract state block from Port: name=vap_outlet. Please add Feed & Product blocks with Arcs to show inlet and outlet stream values in the Stream Table
2025-03-25 16:15:13 [WARNING] idaes.idaes_ui.fv.flowsheet: Port is neither a feed nor a product: name=purge feed-expr=re.compile('^in_|^feed_|^inlet_|^in$|^feed$|^inlet$|_in$|_feed$|_inlet$', re.IGNORECASE) product-expr=re.compile('^out_|^prod_|^outlet_|^out$|^prod$|^outlet$|_out$|_prod$|_outlet$', re.IGNORECASE)
2025-03-

VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x0000018D310B8BE0>, port=62537, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x0000018D310BA1A0>)

2025-03-25 16:15:15 [WARNING] idaes.idaes_ui.fv.flowsheet: Cannot extract state block from Port: name=liq_outlet. Please add Feed & Product blocks with Arcs to show inlet and outlet stream values in the Stream Table
2025-03-25 16:15:15 [WARNING] idaes.idaes_ui.fv.flowsheet: Cannot extract state block from Port: name=vap_outlet. Please add Feed & Product blocks with Arcs to show inlet and outlet stream values in the Stream Table
2025-03-25 16:15:15 [WARNING] idaes.idaes_ui.fv.flowsheet: Port is neither a feed nor a product: name=purge feed-expr=re.compile('^in_|^feed_|^inlet_|^in$|^feed$|^inlet$|_in$|_feed$|_inlet$', re.IGNORECASE) product-expr=re.compile('^out_|^prod_|^outlet_|^out$|^prod$|^outlet$|_out$|_prod$|_outlet$', re.IGNORECASE)
